In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle

In [19]:
filtered_skempi = pd.read_csv('filtered_skempi.csv')

## Create splits

In [ ]:
### Many of the Hold_out_proteins are overlapping, so we need to merge them into super clusters.
hold_out_proteins = filtered_skempi['Hold_out_proteins'].unique()
super_clusters = [set(x.split(',')) for x in hold_out_proteins]
joined=True
while joined:
    joined=False
    new_super_clusters = []
    while super_clusters:
        cur = super_clusters.pop(0)
        for i, cluster in enumerate(super_clusters):
            if cur & cluster:
                cur |= cluster  # Merge the sets
                super_clusters.pop(i)  # Remove the merged set
                joined = True
        new_super_clusters.append(cur)
    super_clusters = new_super_clusters

pdb_to_super_clusters = {}

for cluster in super_clusters:
    for pdb in cluster:
        pdb_to_super_clusters[pdb] = list(cluster)

filtered_skempi['super_clusters'] = [','.join(pdb_to_super_clusters[x[0]]) for x in filtered_skempi['Hold_out_proteins'].str.split(',')]

In [21]:
# filtered_skempi.to_csv('filtered_skempi.csv')

In [22]:
cluster_names = filtered_skempi['super_clusters'].unique()
np.random.seed(17)
np.random.shuffle(cluster_names)
train_size = int(0.7 * len(cluster_names))
test_size = len(cluster_names) - train_size
train_clusters = cluster_names[:train_size]
test_clusters = cluster_names[train_size:]

train_df = filtered_skempi[filtered_skempi['super_clusters'].isin(train_clusters)]
test_df = filtered_skempi[filtered_skempi['super_clusters'].isin(test_clusters)]

print(f'There are {len(train_clusters)} clusters/{len(train_df)} mutants in training')
print(f'There are {len(test_clusters)} clusters/{len(test_df)} mutants in test')

# Save the splits as pickle files
with open('train_clusters.pkl', 'wb') as f:
    pickle.dump(train_clusters, f)

with open('test_clusters.pkl', 'wb') as f:
    pickle.dump(test_clusters, f)

There are 44 clusters/3050 mutants in training
There are 20 clusters/1491 mutants in test
